In [1]:
from __future__ import division
import cympy
import cympy.rm
import pandas
import sys
sys.path.append("D:\\Users\\Jonathan\\Documents\\GitHub\\cymdist")
import function
import lookup
from datetime import datetime
from pytz import timezone
from IPython.display import clear_output
pacific = timezone('US/Pacific')

# Imports useful for graphics
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import DatetimeTickFormatter
output_notebook()

Loading BokehJS ...

In [2]:
# FMU initialize
# Shunt capacitor control
voltage_control = False

# Adding PV on the network
added_PV = False 

# Open a study
filename = 'C:\\Users\\emma\\Documents\\SCE Cymdist\\PIANO_20140718.sxst'
cympy.study.Open(filename)

# Set the grid model with the modification
if voltage_control:
    pass

if added_PV:
    my_pv = function.add_pv("New_PV", "BS2102-L_14036$P5080048-1_14036")

In [3]:
def run_simulation(datetime):
    """
    Args:
        datetime (datetime): datetime to querry database
    Retrun:
        high_voltage, low_voltage, overload, unbalanced (DataFrame)
    """
    # Retrieve microPMU data at t
    inputdt = pacific.localize(datetime)
    upmu_path = '/LBNL/grizzly_bus1/'
    udata = function.get_upmu_data(inputdt, upmu_path)

    # Set the downstream load and voltage and launch load allocation
    function.load_allocation(udata)

    # Run a power flow
    lf = cympy.sim.LoadFlow()
    lf.Run()

    # Get the results
    devices = function.list_devices()
    high_voltage = function.get_high_voltage(devices)
    low_voltage = function.get_low_voltage(devices)
    overload = function.get_overload(devices)
    unbalanced = function.get_unbalanced_line(devices)    

    return high_voltage, low_voltage, overload, unbalanced

put all the result in the same dataframe and save it with json pickle

In [4]:
# Input data
datetimes = [datetime(2016, 8, 25, i, 0, 0) for i in range(0, 24)]

# Result holder
high_voltage = pandas.DataFrame()
low_voltage = pandas.DataFrame()
overload = pandas.DataFrame()
unbalanced = pandas.DataFrame()

for datetime_value in datetimes:
    clear_output()
    print(datetime_value)
    temp_high_voltage, temp_low_voltage, temp_overload, temp_unbalanced = run_simulation(datetime_value)
    
    # Save first row
    high_voltage = pandas.concat([high_voltage, temp_high_voltage.head(1)])
    low_voltage = pandas.concat([low_voltage, temp_low_voltage.head(1)])
    overload = pandas.concat([overload, temp_overload.head(1)])
    unbalanced = pandas.concat([unbalanced, temp_unbalanced.head(1)])

# Add datetime index to the results
high_voltage['datetimes'] = datetimes
high_voltage = high_voltage.set_index('datetimes')

low_voltage['datetimes'] = datetimes
low_voltage = low_voltage.set_index('datetimes')

overload['datetimes'] = datetimes
overload = overload.set_index('datetimes')

unbalanced['datetimes'] = datetimes
unbalanced = unbalanced.set_index('datetimes')

2016-08-25 23:00:00


note:
* minimum voltage excluding value equal to 0
* track device specific ?
* max per phase

need to create the peak at any time but also follow one specific device (plot the one with the biggest sum)

In [64]:
plot = figure(plot_height=500, plot_width=900)
plot.line(high_voltage.index.tolist(), high_voltage.voltage_A.tolist(), legend="phase A", line_width=2, color='blue')
plot.line(high_voltage.index.tolist(), high_voltage.voltage_B.tolist(), legend="phase B", line_width=2, color='red')
plot.line(high_voltage.index.tolist(), high_voltage.voltage_C.tolist(), legend="phase C", line_width=2, color='green')
plot.xaxis.formatter=DatetimeTickFormatter(formats=dict(hours=["%H"]))
plot.xaxis.axis_label = 'Time'
plot.yaxis.axis_label = 'Voltage (pu)'
plot.legend.location = "bottom_right"
show(plot)

In [65]:
plot = figure(plot_height=500, plot_width=900)
plot.line(low_voltage.index.tolist(), low_voltage.voltage_A.tolist(), legend="phase A", line_width=2, color='blue')
plot.line(low_voltage.index.tolist(), low_voltage.voltage_B.tolist(), legend="phase B", line_width=2, color='red')
plot.line(low_voltage.index.tolist(), low_voltage.voltage_C.tolist(), legend="phase C", line_width=2, color='green')
plot.xaxis.formatter=DatetimeTickFormatter(formats=dict(hours=["%H"]))
plot.xaxis.axis_label = 'Time'
plot.yaxis.axis_label = 'Voltage (pu)'
plot.legend.location = "bottom_right"
show(plot)

In [68]:
plot = figure(plot_height=500, plot_width=900)
plot.line(overload.index.tolist(), overload.overload_A.tolist(), legend="phase A", line_width=2, color='blue')
plot.line(overload.index.tolist(), overload.overload_B.tolist(), legend="phase B", line_width=2, color='red')
plot.line(overload.index.tolist(), overload.overload_C.tolist(), legend="phase C", line_width=2, color='green')
plot.xaxis.formatter=DatetimeTickFormatter(formats=dict(hours=["%H"]))
plot.xaxis.axis_label = 'Time'
plot.yaxis.axis_label = 'Overload (%)'
plot.legend.location = "bottom_right"
show(plot)

In [72]:
plot = figure(plot_height=500, plot_width=900)
plot.line(unbalanced.index.tolist(), unbalanced.diff_with_mean.tolist(), legend="difference with mean voltage", line_width=2, color='blue')
plot.xaxis.formatter=DatetimeTickFormatter(formats=dict(hours=["%H"]))
plot.xaxis.axis_label = 'Time'
plot.yaxis.axis_label = 'Unbalanced (%)'
plot.legend.location = "bottom_right"
show(plot)